In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv() # take environment variables from .env

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [11]:
llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-4o-mini',temperature=0.5)

C:\Users\zanab\AppData\Local\Temp\ipykernel_27804\35158059.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY, model_name='gpt-4o-mini',temperature=0.5)


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq":"multiple choice question",
        "options": {
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. Make sure the questions are not repeated and check all the questions to be conforming the text as well. Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [12]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose= True)

C:\Users\zanab\AppData\Local\Temp\ipykernel_27804\3204954895.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key='quiz', verbose= True)


In [13]:
TEMPLATE2 = """
Text: {text}
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students. You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the cognitive and analytical abilites of the students, update the quiz questions which needs to be changed and change the tone such that it perfecty fits the student abilitie.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], 
    template= TEMPLATE
)

In [15]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=['text','number','subject','tone','response_json'], output_variables=["quiz","review"], verbose=True)

In [18]:
file_path = r"C:\Users\zanab\Desktop\Daniel\GenAI\data.txt"

In [19]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [21]:
# Serialize the Python dictionary into a JSON formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [23]:
# How to setup Token Usage Tracking in Langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\zanab\AppData\Local\Temp\ipykernel_27804\2632631866.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons u

In [24]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 2036
Prompt Tokens: 1340
Completion Tokens: 696
Total Cost: 0.0006186


In [25]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive sys

In [27]:
quiz = response.get("quiz")

In [29]:
quiz=json.loads(quiz)

In [30]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices": options, "Correct": correct})

In [31]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Arthur Samuel | c: Walter Pitts | d: Warren McCulloch',
  'Correct': 'b'},
 {'MCQ': "What did Arthur Samuel's earliest machine learning program analyze?",
  'Choices': 'a: Speech patterns | b: Checkers | c: Sonar signals | d: Electrocardiograms',
  'Correct': 'b'},
 {'MCQ': 'Which book introduced a theoretical neural structure in 1949?',
  'Choices': 'a: Learning Machines | b: The Organization of Behavior | c: Artificial Intelligence | d: Pattern Recognition',
  'Correct': 'b'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon?",
  'Choices': 'a: Cybertron | b: NeuralNet | c: PatternMaster | d: LearningBot',
  'Correct': 'a'},
 {'MCQ': 'In what year was a report given about teaching an artificial neural network to recognize characters?',
  'Choices': 'a: 1960 | b: 1973 | c: 1981 | d: 1959',
  'Correct': 'c'}]

In [32]:
pd.DataFrame(quiz_table_data)

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a: Donald Hebb | b: Arthur Samuel | c: Walter ...,b
1,What did Arthur Samuel's earliest machine lear...,a: Speech patterns | b: Checkers | c: Sonar si...,b
2,Which book introduced a theoretical neural str...,a: Learning Machines | b: The Organization of ...,b
3,What was the name of the experimental 'learnin...,a: Cybertron | b: NeuralNet | c: PatternMaster...,a
4,In what year was a report given about teaching...,a: 1960 | b: 1973 | c: 1981 | d: 1959,c
